In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 

In [2]:
df = pd.read_csv('bought_insurance.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=0)

In [4]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [5]:
X_test_scaled

,age,affordibility
2,0.47,1
22,0.40,1
14,0.49,1
17,0.58,1
5,0.56,1
11,0.28,1


In [6]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape = (2,), activation = 'sigmoid', kernel_initializer = 'ones', bias_initializer = 'zeros')
])
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.fit(X_train_scaled, y_train, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 1s 775ms/step - loss: 0.7960 - accuracy: 0.4091
Epoch 2/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7955 - accuracy: 0.4091
Epoch 3/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.7950 - accuracy: 0.4091
Epoch 4/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7944 - accuracy: 0.4091
Epoch 5/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7939 - accuracy: 0.4091
Epoch 6/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.7934 - accuracy: 0.4091
Epoch 7/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7929 - accuracy: 0.4091
Epoch 8/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7923 - accuracy: 0.4091
Epoch 9/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.7918 - accuracy: 0.4091
Epoch 10/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7913 - accuracy: 0.4091
Epoch 11/1000
1/

In [ ]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 222ms/step - loss: 0.4539 - accuracy: 1.0000


[0.4538957178592682, 1.0]

In [ ]:
X_test_scaled

,age,affordibility
2,0.47,1
22,0.40,1
14,0.49,1
17,0.58,1
5,0.56,1
11,0.28,1


In [ ]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 122ms/step


array([[0.6177279],
       [0.5396322],
       [0.6391427],
       [0.727986 ],
       [0.7094467],
       [0.4033525]], dtype=float32)

In [ ]:
y_test

2     1
22    1
14    1
17    1
5     1
11    0
Name: bought_insurance, dtype: int64

In [ ]:
coef, intercept = model.get_weights()
coef, intercept

(array([[4.586479 ],
        [1.1000606]], dtype=float32),
 array([-2.7757905], dtype=float32))

In [ ]:
def sigmoid(x):
  import math
  return 1 / (1 + math.exp(-x))

In [ ]:
def prediction_function(age, affordibility):
  weighted_sum = coef[0] * age + coef[1] * affordibility + intercept
  return sigmoid(weighted_sum)

In [ ]:
prediction_function(0.47 ,1)

0.617727892902254

In [ ]:
prediction_function(0.40 ,1)

0.5396321538833132

In [ ]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1 - epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    
    return -np.mean(y_true * np.log(y_predicted_new) + (1 - y_true) * np.log(1 - y_predicted_new))

In [ ]:
def sigmoid_numpy(x):
  return 1 / (1 + np.exp(-x))

sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [ ]:
class myNN:
    def __init__(self): # Here is my constructor
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold): # Here is my fit method in function, self is always firs argument
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_thresold) # It will return me a tuple
        print(f"Final weights and bias : w1: {self.w1}, w2: {self.w2}, bias: {self.bias}") # So my gradient_descent will return my weights an bias 
        
    def predict(self, X_test): # Its my prediction function
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias # Its my weighted_sum from pattern  
        return sigmoid_numpy(weighted_sum) # And its activated by sigmoid_numpy

    def gradient_descent(self, age, affordability, y_true, epochs, loss_thresold): # This function help finding weights
        w1 = w2 = 1
        bias = 0
        learning_rate = 0.5
       
        n = len(age)
        
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1 / n) * np.dot(np.transpose(age), (y_predicted - y_true)) 
            w2d = (1 / n) * np.dot(np.transpose(affordability), (y_predicted - y_true)) 
            bias_d = np.mean(y_predicted - y_true)
            
            w1 = w1 - learning_rate * w1d
            w2 = w2 - learning_rate * w2d
            bias = bias - learning_rate * bias_d
            
            if i%50 == 0: # The lines will be printing every fiftieth epoch
                print(f'Epoch : {i}, w1 : {w1}, w2 : {w2}, bias : {bias}, loss : {loss}')
            
            if loss <= loss_thresold:
                print(f'Epoch : {i}, w1 : {w1}, w2 : {w2}, bias : {bias}, loss : {loss}')
                break

        return w1, w2, bias
  

In [ ]:
customModel = myNN() # I create object of my class 
customModel.fit(X_train_scaled, y_train, epochs = 500, loss_thresold = 0.4606) # Here is my fit method

Epoch:0, w1:0.9644915668168719, w2:0.9224250019701856, bias:-0.1521744437835525, loss:0.7960362503355486
Epoch:50, w1:1.5553744970498753, w2:0.8974557509174909, bias:-1.4309411203747453, loss:0.562365515890499
Epoch:100, w1:2.344788791747113, w2:1.0539002051324673, bias:-1.8614004721522848, loss:0.5287372092356005
Epoch:150, w1:3.0622456983139172, w2:1.1183996759491954, bias:-2.1962899781520844, loss:0.5033566832422639
Epoch:200, w1:3.7072113493477024, w2:1.1501309696389928, bias:-2.478570705307821, loss:0.4833975777974661
Epoch:250, w1:4.285628216525323, w2:1.1704318345778901, bias:-2.7263829812913447, loss:0.46747326254088184
Epoch:276, w1:4.562607017100527, w2:1.1793032507081156, bias:-2.844701939090328, loss:0.4604647723587135
Final weights and bias: w1: 4.562607017100527, w2: 1.1793032507081156, bias: -2.844701939090328


In [ ]:
coef, intercept # It is comaparation with my keras model

(array([[4.586479 ],
        [1.1000606]], dtype=float32),
 array([-2.7757905], dtype=float32))

In [ ]:
customModel.predict(X_test_scaled) # And here is prediction comparation with my keras model

2     0.617518
22    0.539826
14    0.638827
17    0.727289
5     0.708820
11    0.404230
dtype: float64

In [ ]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 134ms/step


array([[0.6177279],
       [0.5396322],
       [0.6391427],
       [0.727986 ],
       [0.7094467],
       [0.4033525]], dtype=float32)